# Deep Convolutional GAN 2
- 실습으로 배우기

In [2]:
# 실행결과
# - 필요 라이브러리 설치
# - GAN 학습을 위한 기본적인 환경 구축

# 필요한 라이브러리를 임포트합니다.
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import time
import random
import glob
# from google.colab import drive # 구글 코랩에서 구글 드라이브를 사용하기 위해 필요합니다.
from torchvision.datasets import ImageFolder # 이미지 데이터셋을 로드하기 위해 필요합니다.
# from google.colab import files # 코랩에서 파일을 다운로드하기 위해 필요합니다.
from IPython.display import display, HTML # 주피터/코랩 환경에서 결과를 시각화하기 위해 필요합니다.

# # 구글 드라이브를 마운트합니다. 데이터셋이나 체크포인트를 저장/로드하는 데 사용될 수 있습니다.
# try:
#     drive.mount('/content/drive')
#     print("구글 드라이브 마운트 성공!")
# except:
#     print("구글 드라이브 마운트 실패 또는 마운트 취소")

In [3]:
# 실행 결과
# - 재현성 확보 및 장치 설정을 위한 초기 설정
# 재현 가능한 결과를 위해 랜덤 시드를 설정하는 함수입니다.
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available(): # CUDA를 사용할 수 있는 경우 GPU 시드도 설정합니다.
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True # 결정론적 CUDA 연산을 사용합니다.

# 시드를 42로 설정합니다.
set_seed(42)

# 학습에 사용할 장치를 설정합니다 (CUDA가 사용 가능하면 GPU, 아니면 CPU).
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

사용 장치: cpu


In [4]:
# 학습에 사용될 하이퍼파라미터 및 모델 설정을 정의합니다.
batch_size = 64 # 한 번의 학습 iteration에 사용할 이미지 수
image_size = 64 # 이미지 크기 (64x64 픽셀)
nz = 100 # 생성자의 입력으로 사용될 노이즈 벡터의 크기
ngf = 64 # 생성자의 첫 번째 컨볼루션 계층의 필터 수 (채널 수)
ndf = 64 # 판별자의 첫 번째 컨볼루션 계층의 필터 수 (채널 수)
num_epochs = 50 # 전체 데이터셋을 반복할 횟수
lr = 0.0003 # 학습률 (learning rate)
beta1 = 0.5 # Adam optimizer의 베타1 파라미터

In [6]:
# CIFAR-10 데이터셋에서 강아지 이미지만 다운로드하고 추출하는 함수입니다.
def download_dog_dataset():
    print("강아지 데이터셋 다운로드 중...")

    import torchvision.datasets as datasets # 데이터셋 다운로드를 위해 torchvision.datasets을 임포트합니다.

    # CIFAR-10 데이터셋을 다운로드합니다. train=True는 학습 데이터셋을 다운로드함을 의미합니다.
    cifar10 = datasets.CIFAR10(root='./cifar10', download=True, train=True)

    # CIFAR-10 클래스 목록을 확인합니다.
    class_labels = cifar10.classes
    print(f"CIFAR-10 클래스 목록: {class_labels}")

    # 'dog' 클래스의 인덱스를 찾습니다.
    dog_idx = class_labels.index('dog')
    print(f"강아지 클래스 인덱스: {dog_idx}")

    # CIFAR-10 데이터셋에서 강아지 이미지만 추출합니다.
    dog_images = []

    for i in range(len(cifar10)):
        img, label = cifar10[i]
        if label == dog_idx: # 라벨이 강아지 인덱스와 같으면 이미지를 추가합니다.
            dog_images.append(img)

    print(f"{len(dog_images)}개의 강아지 이미지를 추출했습니다.")

    # 추출한 강아지 이미지를 저장할 폴더를 생성합니다.
    os.makedirs('./dog_dataset/dogs', exist_ok=True)

    # 추출된 강아지 이미지를 지정된 폴더에 JPEG 형식으로 저장합니다.
    for i, img in enumerate(dog_images):
        img.save(f'./dog_dataset/dogs/dog_{i}.jpg')

    print(f"이미지를 './dog_dataset/dogs/' 폴더에 저장했습니다.")
    return './dog_dataset' # 저장된 데이터셋의 루트 경로를 반환합니다.

In [8]:
# 데이터셋을 확인하고 필요한 경우 다운로드한 후 데이터 로더를 설정합니다.
try:
    # './dog_dataset' 폴더가 존재하고 이미지 파일이 있는지 확인합니다.
    if not os.path.exists('./dog_dataset') or len(glob.glob('./dog_dataset/*/*.jpg')) == 0:
        data_root = download_dog_dataset() # 데이터셋이 없으면 다운로드합니다.
    else:
        data_root = './dog_dataset' # 데이터셋이 있으면 기존 데이터셋을 사용합니다.
        print(f"기존 데이터셋 사용: {data_root}")
except:
    # 데이터셋 확인 중 오류가 발생하면 다운로드를 시도합니다.
    print("데이터셋 확인 중 오류 발생, 다운로드를 시도합니다.")
    data_root = download_dog_dataset()

# 이미지 변환(Transform)을 정의합니다.
# - Resize: 이미지 크기를 image_size x image_size로 조절합니다.
# - CenterCrop: 중앙 부분을 image_size x image_size로 잘라냅니다.
# - ToTensor: PIL 이미지를 PyTorch Tensor로 변환합니다.
# - Normalize: 이미지 픽셀 값을 [-1, 1] 범위로 정규화합니다.
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ImageFolder를 사용하여 데이터셋을 로드합니다. ImageFolder는 폴더 구조를 기반으로 데이터셋을 구성합니다.
try:
    dataset = ImageFolder(root=data_root, transform=transform)
    # 데이터 로더를 생성합니다. 학습 시 데이터를 효율적으로 배치 단위로 불러오는 역할을 합니다.
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    print(f"데이터셋 로드 완료: {len(dataset)} 이미지")
except Exception as e:
    # 데이터셋 로드 중 오류가 발생하면 예외를 처리하고 임의의 데이터셋을 생성합니다.
    print(f"데이터셋 로드 오류: {e}")
    print("임의 데이터로 코드를 계속 실행합니다...")

    # 임의 데이터셋을 생성하는 함수입니다.
    def create_random_dataset(num_samples=1000):
        random_data = torch.randn(num_samples, 3, image_size, image_size) # 랜덤 노이즈로 이미지 형태의 데이터를 생성합니다.
        random_data = torch.clamp((random_data * 0.2) + 0.5, 0, 1)  # 값 범위를 [0, 1]로 조정합니다.
        random_dataset = [(img, 0) for img in random_data]  # (이미지, 라벨) 형태의 리스트를 만듭니다. (라벨은 임의로 0으로 설정)
        return random_dataset

    # 사용자 정의 RandomDataset 클래스입니다. DataLoader에 사용하기 위해 PyTorch Dataset을 상속합니다.
    class RandomDataset(torch.utils.data.Dataset):
        def __init__(self, data):
            self.data = data

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            return self.data[idx] # 인덱스에 해당하는 데이터를 반환합니다.

    random_dataset = RandomDataset(create_random_dataset()) # 임의 데이터셋 객체를 생성합니다.
    # 임의 데이터셋을 위한 데이터 로더를 생성합니다.
    dataloader = DataLoader(random_dataset, batch_size=batch_size, shuffle=True)
    print(f"임의 데이터셋 생성 완료: {len(random_dataset)} 이미지")

기존 데이터셋 사용: ./dog_dataset
데이터셋 로드 완료: 5000 이미지


In [9]:
# 모델 가중치를 초기화하는 함수입니다. DCGAN 논문의 권장 사항을 따릅니다.
def weights_init(m):
    classname = m.__class__.__name__
    # Conv 계층의 가중치를 평균 0.0, 표준편차 0.02인 정규분포로 초기화합니다.
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    # BatchNorm 계층의 가중치를 평균 1.0, 표준편차 0.02인 정규분포로 초기화하고, bias는 0으로 초기화합니다.
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# 생성자 (Generator) 모델을 정의합니다.
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # Sequential 모델은 계층들을 순차적으로 연결합니다.
        self.main = nn.Sequential(
            # ConvTranspose2d (Deconvolution) 계층을 사용하여 노이즈 벡터에서 이미지로 크기를 키웁니다.
            # nz: 입력 채널 수 (노이즈 벡터 크기)
            # ngf * 8: 출력 채널 수
            # 4: 커널 크기
            # 1: 스트라이드
            # 0: 패딩
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8), # 배치 정규화 계층
            nn.ReLU(True), # ReLU 활성화 함수

            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            # 마지막 계층: 3채널 (RGB) 이미지 출력을 위해 스트라이드 2를 사용합니다.
            nn.ConvTranspose2d(ngf, 3, 4, 2, 1, bias=False),
            nn.Tanh() # Tanh 활성화 함수를 사용하여 출력 픽셀 값을 [-1, 1] 범위로 만듭니다.
        )

    # forward 메서드는 입력 텐서를 받아 모델을 통과시킨 결과를 반환합니다.
    def forward(self, input):
        return self.main(input)


# 판별자 (Discriminator) 모델을 정의합니다.
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Conv2d 계층을 사용하여 이미지를 입력받아 특징을 추출하고 크기를 줄입니다.
            # 3: 입력 채널 수 (RGB 이미지)
            # ndf: 출력 채널 수
            # 4: 커널 크기
            # 2: 스트라이드
            # 1: 패딩
            nn.Conv2d(3, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True), # LeakyReLU 활성화 함수

            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),

            # 마지막 계층: 이미지의 진짜/가짜를 판별하기 위해 1개의 출력 채널을 사용합니다.
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid() # Sigmoid 활성화 함수를 사용하여 출력을 [0, 1] 범위의 확률 값으로 만듭니다.
        )

    # forward 메서드는 입력 텐서를 받아 판별 결과를 반환합니다.
    # .view(-1, 1).squeeze(1)는 출력을 1차원 텐서로 변환합니다.
    def forward(self, input):
        return self.main(input).view(-1, 1).squeeze(1)


# 생성자와 판별자 모델 객체를 생성하고 지정된 장치(GPU 또는 CPU)로 이동시킵니다.
netG = Generator().to(device)
netD = Discriminator().to(device)

# 모델 가중치를 초기화합니다.
netG.apply(weights_init)
netD.apply(weights_init)

# 모델 구조를 출력합니다.
print("생성자 모델 구조:")
print(netG)
print("\n판별자 모델 구조:")
print(netD)

# 손실 함수로 Binary Cross Entropy (BCELoss)를 사용합니다. GAN 학습에서 주로 사용됩니다.
criterion = nn.BCELoss()

# 학습 과정 중 생성 성능을 확인하기 위한 고정된 노이즈 벡터를 생성합니다.
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# 진짜 이미지와 가짜 이미지에 대한 라벨을 설정합니다.
real_label = 1
fake_label = 0

# 생성자와 판별자를 위한 Adam optimizer를 설정합니다.
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

생성자 모델 구조:
Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Ta

In [10]:
# 이미지를 시각화하는 함수입니다.
def visualize_images(images, title=None, display_in_notebook=True):
    # 이미지를 [-1, 1] 범위에서 [0, 1] 범위로 변환합니다.
    images = (images + 1) / 2.0

    # 이미지 그리드를 만듭니다. 여러 이미지를 한 번에 시각화하는 데 사용됩니다.
    grid = torchvision.utils.make_grid(images, padding=2, normalize=False)
    # PyTorch 텐서를 Matplotlib에서 표시할 수 있는 numpy 배열로 변환합니다. 채널 순서를 변경합니다.
    img = grid.permute(1, 2, 0).cpu().numpy()

    # 이미지를 플로팅합니다.
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    # 제목이 있으면 제목을 설정합니다.
    if title:
        plt.title(title)
    # 축 정보를 표시하지 않습니다.
    plt.axis('off')

    # 노트북 환경에서 이미지를 표시할지 여부를 결정합니다.
    if display_in_notebook:
        plt.show()

    return img # 시각화된 이미지의 numpy 배열을 반환합니다.

# 결과를 저장할 폴더와 체크포인트를 저장할 폴더를 생성합니다.
os.makedirs("results", exist_ok=True)
os.makedirs("checkpoints", exist_ok=True)

# 학습 중 생성자와 판별자의 손실 값을 저장할 리스트입니다.
G_losses = []
D_losses = []
# 각 에폭의 마지막에 생성된 이미지를 저장할 리스트입니다.
img_list = []

In [ ]:
print("학습 시작...")

# 학습 진행 상황을 보여주는 간단한 프로그레스 바 함수입니다.
def progress_bar(current, total, bar_length=50):
    fraction = current / total # 전체 중 현재 진행된 비율
    arrow = int(fraction * bar_length) * '=' # 진행된 부분을 나타내는 '=' 문자열
    padding = (bar_length - len(arrow)) * ' ' # 남은 부분을 나타내는 공백 문자열
    return f"[{arrow}{padding}] {int(fraction * 100)}%" # 프로그레스 바 문자열 반환

# GAN 학습 반복문입니다. num_epochs 만큼 반복합니다.
for epoch in range(num_epochs):
    start_time = time.time() # 각 에폭의 시작 시간을 기록합니다.
    # 데이터 로더에서 데이터를 배치 단위로 불러옵니다.
    for i, data in enumerate(dataloader, 0):
        # 판별자 네트워크의 그래디언트를 0으로 초기화합니다.
        netD.zero_grad()
        # 로드된 데이터가 리스트 형태인지 확인하고 실제 이미지를 가져옵니다.
        if isinstance(data, list) and len(data) == 2:
            real_cpu = data[0].to(device) # 첫 번째 요소가 이미지 데이터라고 가정하고 장치로 이동합니다.
        else:
            real_cpu = data[0].to(device) # 이미지가 바로 첫 번째 요소라고 가정하고 장치로 이동합니다.

        batch_size = real_cpu.size(0) # 현재 배치의 크기를 가져옵니다.
        # 진짜 이미지에 대한 라벨 텐서를 생성합니다.
        label = torch.full((batch_size,), real_label, dtype=torch.float, device=device)

        # 진짜 이미지를 판별자 네트워크에 통과시킵니다.
        output = netD(real_cpu)
        # 진짜 이미지에 대한 판별자의 손실을 계산합니다. 판별자는 진짜 이미지를 1로 예측해야 합니다.
        errD_real = criterion(output, label)
        # 역전파를 통해 판별자의 그래디언트를 계산합니다.
        errD_real.backward()
        D_x = output.mean().item() # 판별자가 진짜 이미지를 진짜로 예측한 평균 확률을 기록합니다.

        # 생성자 네트워크의 입력으로 사용할 노이즈 벡터를 생성합니다.
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        # 생성자 네트워크를 사용하여 가짜 이미지를 생성합니다.
        fake = netG(noise)
        # 가짜 이미지에 대한 라벨 텐서를 생성합니다.
        label.fill_(fake_label)

        # 가짜 이미지를 판별자 네트워크에 통과시킵니다. .detach()를 사용하여 생성자 쪽으로 그래디언트가 흐르지 않도록 합니다.
        output = netD(fake.detach())
        # 가짜 이미지에 대한 판별자의 손실을 계산합니다. 판별자는 가짜 이미지를 0으로 예측해야 합니다.
        errD_fake = criterion(output, label)
        # 역전파를 통해 판별자의 그래디언트를 계산합니다.
        errD_fake.backward()
        D_G_z1 = output.mean().item() # 판별자가 가짜 이미지를 진짜로 예측한 평균 확률 (생성자 학습 전에 기록)

        # 판별자의 총 손실은 진짜 이미지 손실과 가짜 이미지 손실의 합입니다.
        errD = errD_real + errD_fake
        # 판별자 네트워크의 가중치를 업데이트합니다.
        optimizerD.step()

        # 생성자 네트워크의 그래디언트를 0으로 초기화합니다.
        netG.zero_grad()
        # 생성자 학습을 위해 가짜 이미지에 대한 라벨을 '진짜' (1)로 설정합니다. 생성자는 판별자가 가짜 이미지를 진짜로 착각하게 만들려고 학습합니다.
        label.fill_(real_label)

        # 생성자가 만든 가짜 이미지를 다시 판별자 네트워크에 통과시킵니다.
        output = netD(fake)
        # 생성자의 손실을 계산합니다. 생성자는 판별자가 가짜 이미지를 진짜로 판단하도록 유도하므로 라벨을 1로 사용합니다.
        errG = criterion(output, label)
        # 역전파를 통해 생성자의 그래디언트를 계산합니다.
        errG.backward()
        D_G_z2 = output.mean().item() # 판별자가 가짜 이미지를 진짜로 예측한 평균 확률 (생성자 학습 후에 기록)

        # 생성자 네트워크의 가중치를 업데이트합니다.
        optimizerG.step()

        # 학습 중 손실 값을 기록합니다.
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # 10 배치마다 학습 진행 상황을 출력합니다.
        if i % 10 == 0:
            prog = progress_bar(i, len(dataloader)) # 프로그레스 바 문자열 생성
            print(f'\r에폭 [{epoch+1}/{num_epochs}] 배치 {prog} '
                  f'Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f} '
                  f'D(x): {D_x:.4f} D(G(z)): {D_G_z1:.4f}/{D_G_z2:.4f}', end='') # 현재 라인에 출력하고 줄바꿈하지 않음

    print(' ') # 에폭의 끝에서 줄바꿈합니다.

    # 각 에폭의 끝에서 고정된 노이즈로 이미지를 생성하여 학습 진행 상황을 시각적으로 확인합니다.
    with torch.no_grad(): # 그래디언트 계산을 비활성화합니다.
        fake = netG(fixed_noise).detach().cpu() # 가짜 이미지를 생성하고 CPU로 이동시킵니다.
    img_list.append(fake) # 생성된 이미지를 리스트에 추가합니다.

    # 생성된 이미지를 시각화하고 저장합니다.
    img = visualize_images(fake, title=f'epoch {epoch+1} Result')

    # 결과 이미지를 파일로 저장합니다.
    plt.savefig(f'results/fake_dogs_epoch_{epoch+1}.png')
    plt.close() # 현재 플롯을 닫아 메모리를 확보합니다.

    # 5 에폭마다 또는 마지막 에폭에서 모델 체크포인트를 저장합니다.
    if (epoch + 1) % 5 == 0 or (epoch + 1) == num_epochs:
        torch.save({
            'generator': netG.state_dict(), # 생성자 모델의 가중치
            'discriminator': netD.state_dict(), # 판별자 모델의 가중치
            'optimizerG': optimizerG.state_dict(), # 생성자 옵티마이저 상태
            'optimizerD': optimizerD.state_dict(), # 판별자 옵티마이저 상태
            'epoch': epoch, # 현재 에폭 번호
            'G_losses': G_losses, # 생성자 손실 기록
            'D_losses': D_losses, # 판별자 손실 기록
        }, f'checkpoints/gan_model_epoch_{epoch+1}.pth') # 저장할 파일 경로
        print(f"모델 체크포인트 저장: 에폭 {epoch+1}")

    # 에폭별 소요 시간을 계산하고 출력합니다.
    elapsed = time.time() - start_time
    print(f'에폭 {epoch+1} 완료, 소요 시간: {elapsed:.2f}초')

print("학습 완료!")

학습 시작...
에폭 [1/50] 배치 [===============================                   ] 63% Loss_D: 2.3174 Loss_G: 14.1506 D(x): 0.9179 D(G(z)): 0.8612/0.0000

In [ ]:
# 생성자와 판별자의 손실 그래프를 그립니다.
plt.figure(figsize=(10, 5))
plt.title("Loss of Generator and Discriminator")
plt.plot(G_losses, label="Generator") # 생성자 손실 플롯
plt.plot(D_losses, label="Discriminator") # 판별자 손실 플롯
plt.xlabel("Repeat") # x축 레이블
plt.ylabel("Loss") # y축 레이블
plt.legend() # 범례 표시
plt.savefig('results/loss_plot.png') # 그래프를 파일로 저장합니다.
plt.show() # 그래프를 화면에 표시합니다.
plt.close() # 현재 플롯을 닫습니다.

# 각 에폭의 마지막에 생성된 이미지들을 그리드로 시각화합니다.
plt.figure(figsize=(12, 12))
# 그리드의 행과 열 수를 계산합니다.
rows = int(np.sqrt(num_epochs))
cols = int(np.ceil(num_epochs / rows))

# 각 에폭별로 생성된 이미지 중 일부(16개)를 서브플롯으로 표시합니다.
for i in range(min(num_epochs, len(img_list))): # 학습 에폭 또는 저장된 이미지 수 중 작은 값만큼 반복
    plt.subplot(rows, cols, i + 1) # 서브플롯 위치 설정
    plt.axis('off') # 축 숨기기
    plt.title(f'epoch {i+1}') # 서브플롯 제목 (에폭 번호)

    if i < len(img_list):
        # 저장된 이미지 리스트에서 해당 에폭의 이미지를 가져와 그리드로 만듭니다.
        img = torchvision.utils.make_grid(img_list[i][:16], padding=2, normalize=True)
        # 이미지를 Matplotlib에서 표시할 수 있는 형태로 변환하고 표시합니다.
        plt.imshow(np.transpose(img.cpu().numpy(), (1, 2, 0)))

plt.tight_layout() # 서브플롯 간의 간격을 자동으로 조정합니다.
plt.savefig('results/progress.png') # 이미지 그리드를 파일로 저장합니다.
plt.show() # 이미지 그리드를 화면에 표시합니다.


# 학습이 완료된 생성자 모델을 사용하여 새로운 이미지를 생성하는 함수입니다.
def generate_new_dogs(num_images=16):
    netG.eval() # 생성자 모델을 평가 모드로 설정합니다 (드롭아웃 등 비활성화).

    with torch.no_grad(): # 그래디언트 계산을 비활성화합니다.
        # 새로운 노이즈 벡터를 생성합니다.
        noise = torch.randn(num_images, nz, 1, 1, device=device)

        # 생성자 모델을 사용하여 새로운 가짜 이미지를 생성하고 CPU로 이동시킵니다.
        fake_dogs = netG(noise).detach().cpu()

        # 생성된 새로운 이미지들을 시각화하고 저장합니다.
        visualize_images(fake_dogs, title=f'Generated Dog Image {num_images}')
        plt.savefig('results/final_generated_dogs.png') # 최종 생성 이미지 저장

        print(f"{num_images}개의 새로운 강아지 이미지를 생성했습니다. ('results/final_generated_dogs.png'에 저장)")

        return fake_dogs # 생성된 이미지 데이터를 반환합니다.

print("\n최종 모델로 새 이미지 생성:")
generate_new_dogs(16) # 최종 모델로 16개의 새로운 이미지를 생성합니다.

# 생성된 결과 파일 다운로드 방법을 안내하는 함수입니다.
def download_results():
    print("\n생성된 결과 다운로드:")
    print("1. 왼쪽 사이드바에서 파일 탭(📁)을 클릭합니다.")
    print("2. 'results' 폴더에서 생성된 이미지를 다운로드합니다.")
    print("또는 아래 코드를 실행하여 결과 파일을 다운로드할 수 있습니다:")
    print("files.download('results/final_generated_dogs.png')")
    print("files.download('results/progress.png')")
    print("files.download('results/loss_plot.png')")

print("\n=== GAN 실습2 완료 ===")
print("생성된 모든 결과는 'results' 폴더에 저장되었습니다.")
download_results() # 결과 다운로드 방법을 안내합니다.